# Import Dependencies


In [ ]:
# Import necessary functions from the uploaded files
import os
from final_response_front_end_main import initialize_system, process_query
from user_history_utils import save_chat_pkl_by_embedding
from user_history_utils import save_chat_json

# Intialize Environment (do this once)

In [2]:
# Initialize the environment
DEVICE, TOKENIZER, EMBEDDING_MODEL, LLM_MODEL, LLM_SYSTEM_PROMPT, QDRANT_CLIENT, CHUNK_COLLECTION, HISTORY_COLLECTION, BM25_SEARCH_FUNCTION, user_query_state_history, query_num, HISTORICAL_QUERY_NUM = initialize_system()

qdrant_vector_store/local_embedding_models/Snowflake/snowflake-arctic-embed-l-v2.0
Loading Snowflake/snowflake-arctic-embed-l-v2.0 from local storage...


Some weights of XLMRobertaModel were not initialized from the model checkpoint at qdrant_vector_store/local_embedding_models/Snowflake/snowflake-arctic-embed-l-v2.0 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Snowflake/snowflake-arctic-embed-l-v2.0 embedding model loaded to cuda


############### System Prompt:

    You are an advanced AI research assistant. Generate detailed and comprehensive responses that supplement students' and academic researchers' work with information grounded in highly cited AI/ML research papers, specifically in fields like NLP and CV. The response should not focus on one area of study but should be informed by both the current query and chat history to generate a well-rounded answer.

    1. **Introductory Overview**: Start with a high-level conceptual overview of the topic, providing a brief and clear explanation that covers the essential aspects of the subject. This should be accessible to a broad audience.

    2. **Technical Overview**: After the conceptual overview, provide a more in-depth, technical explanation that dives deeper into the topic. This could include relevant algorithms, methods, or models, as well as their theoretical foundations.

    3. **Ex

# Call Response Generation Function (do this as much as you like)

In [ ]:
def call_back_end_response_generation(
                                      user_query_state_history: dict,
                                      DESIRED_HISTORY_WINDOW_SIZE: int,
                                      DESIRED_CONTEXT_CHUNKS_TOP_K: int,
                                      RAG_SWITCH: bool,
                                      HISTORY_SWITCH: bool,
                                      BM25_SWITCH: bool,
                                      TOPIC_RETRIEVAL_SWITCH: bool,
                                      HISTORIC_QUERY_SIMILARITY_THRESHOLD: float, # [0, 1] range (filter)
                                      QUERY_TEXT: str,
                                      QUERY_NUM: int,
                                      final_json_path: str
    ):

    # Call the `process_query()` function with the inputs
    user_query_state_history[QUERY_NUM] = process_query(
        DESIRED_HISTORY_WINDOW_SIZE, 
        DESIRED_CONTEXT_CHUNKS_TOP_K, 
        RAG_SWITCH, 
        HISTORY_SWITCH, 
        BM25_SWITCH, 
        TOPIC_RETRIEVAL_SWITCH, 
        HISTORIC_QUERY_SIMILARITY_THRESHOLD, 
        QUERY_TEXT, 
        user_query_state_history,
        QUERY_NUM, 
        QDRANT_CLIENT, 
        CHUNK_COLLECTION,
        HISTORY_COLLECTION,
        LLM_MODEL,
        LLM_SYSTEM_PROMPT,
        DEVICE,
        EMBEDDING_MODEL, 
        TOKENIZER,
        BM25_SEARCH_FUNCTION
    )

    return user_query_state_history

# Input Test Query Processing Function

In [ ]:

def process_list_of_queries_for_testing(
                            saved_chats_topic_name, # The name of the folder to save questions in
                            list_of_questions, # The list of questions being used to get responses
                            task_folder='experiments',
                            user_output_folder='my_experiment', # the folder to save the chats in
                            DESIRED_HISTORY_WINDOW_SIZE=0, # The lookback window size for history utilization
                            DESIRED_CONTEXT_CHUNKS_TOP_K=5, # The number of chunks requested for a question
                            RAG_SWITCH=True, # Front-end user input for enabling RAG
                            HISTORY_SWITCH=False, # Front-end user input for enabling history usage
                            BM25_SWITCH=False, # Front-end user input for enabling BM25 search
                            TOPIC_RETRIEVAL_SWITCH=False, # Front-end user input for enabling topic retrieval
                            HISTORIC_QUERY_SIMILARITY_THRESHOLD=0.3 # History query-to-query similarity filter min bound
    ):

    directory = f'user_output/{task_folder}/{user_output_folder}/{saved_chats_topic_name}'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Define the name of the chat, and the .json / .pkl files that will be saved with it
    chat_json_name = f"{saved_chats_topic_name}.json"
    chat_embedded_history_name = 'user_embedded_history.pkl'
    chat_non_embedded_history_name = 'user_non_embedded_history.pkl'

    # Define the path of the json / pkl files
    final_json_path = os.path.join(directory, chat_json_name)
    final_embedded_history_path = os.path.join(directory, chat_embedded_history_name)
    final_non_embedded_history_path = os.path.join(directory, chat_non_embedded_history_name)

    # Define the initial user state
    user_state = {}

    for i, question in enumerate(list_of_questions):
        query_num = i + 1
        # Your processing logic goes here (e.g., semantic search, BM25, RAG)
        print(f"Processing query {query_num}: {question}\n")
        print(f"RAG Switch: {RAG_SWITCH}")
        print(f"History Switch: {HISTORY_SWITCH}")
        print(f"BM25 Switch: {BM25_SWITCH}")
        print(f"Topic Retrieval Switch: {TOPIC_RETRIEVAL_SWITCH}\n")

        user_state = call_back_end_response_generation(
                                          user_state,
                                          DESIRED_HISTORY_WINDOW_SIZE,
                                          DESIRED_CONTEXT_CHUNKS_TOP_K,
                                          RAG_SWITCH,
                                          HISTORY_SWITCH,
                                          BM25_SWITCH,
                                          TOPIC_RETRIEVAL_SWITCH,
                                          HISTORIC_QUERY_SIMILARITY_THRESHOLD,
                                          QUERY_TEXT=question,
                                          final_json_path=final_json_path,
                                          QUERY_NUM=query_num
        )

        save_chat_json(user_state[query_num], file_path=final_json_path)

        print(f"Response: {user_state[query_num]['response_text']}\n")

    # After processing the query, you might want to save the results to disk or perform further actions
    save_chat_pkl_by_embedding(user_state, 
                                embedded_path=final_embedded_history_path,
                                non_embedded_path=final_non_embedded_history_path
    )

    print(f'Final User State Length: {len(user_state)}')

    return #user_state

# Experiment Scheduler

In [13]:
my_input_questions = [
'Please explain batch normalization.',                                    # 1
'How does it relate to layer normalization?',                             # 2
'Can you explain the advantages of each method?']                         # 3
# 'What are some real-world applications of batch normalization?',          # 4
# 'How does batch normalization affect the performance of neural networks?',# 5
# 'What are the key differences between normalization techniques?',         # 6
# #Hard topic shift begins here:
# 'What is reinforcement learning and how does it work?',                   # 7
# 'Can you explain the exploration-exploitation tradeoff?',                 # 8
# 'What are some popular algorithms used in reinforcement learning?',       # 9
# 'asdfadsgfasd',                                                           # 10
# 'exit'                                                                    # end
# ]

at_top_k_chunks = 5 # @ the number of chunks in the system

zero_shot_experiments_at_5_top_k = {

    # experiment_1: Naive response without retrieval
    'experiment_1' : {
        'DESIRED_HISTORY_WINDOW_SIZE': 0, # The lookback window size for history utilization
        'DESIRED_CONTEXT_CHUNKS_TOP_K': at_top_k_chunks, # The number of chunks requested for a question
        'RAG_SWITCH': False, # Front-end user input for enabling RAG
        'HISTORY_SWITCH': False, # Front-end user input for enabling history usage
        'BM25_SWITCH': False, # Front-end user input for enabling BM25 search
        'TOPIC_RETRIEVAL_SWITCH': False, # Front-end user input for enabling topic retrieval
        'HISTORIC_QUERY_SIMILARITY_THRESHOLD': 0.3
    },

    # experiment_2: RAG response with basic retrieval
    'experiment_2' : {
        'DESIRED_HISTORY_WINDOW_SIZE': 0, # The lookback window size for history utilization
        'DESIRED_CONTEXT_CHUNKS_TOP_K': at_top_k_chunks, # The number of chunks requested for a question
        'RAG_SWITCH': True, # Front-end user input for enabling RAG
        'HISTORY_SWITCH': False, # Front-end user input for enabling history usage
        'BM25_SWITCH': False, # Front-end user input for enabling BM25 search
        'TOPIC_RETRIEVAL_SWITCH': False, # Front-end user input for enabling topic retrieval
        'HISTORIC_QUERY_SIMILARITY_THRESHOLD': 0.3
    },

    # experiment_3: RAG response hybrid retrieval
    'experiment_3' : {
        'DESIRED_HISTORY_WINDOW_SIZE': 0, # The lookback window size for history utilization
        'DESIRED_CONTEXT_CHUNKS_TOP_K': at_top_k_chunks, # The number of chunks requested for a question
        'RAG_SWITCH': True, # Front-end user input for enabling RAG
        'HISTORY_SWITCH': False, # Front-end user input for enabling history usage
        'BM25_SWITCH': True, # Front-end user input for enabling BM25 search
        'TOPIC_RETRIEVAL_SWITCH': False, # Front-end user input for enabling topic retrieval
        'HISTORIC_QUERY_SIMILARITY_THRESHOLD': 0.3
    },

    # experiment_4: RAG response hybrid with topic level retrieval
    'experiment_4' : {
        'DESIRED_HISTORY_WINDOW_SIZE': 0, # The lookback window size for history utilization
        'DESIRED_CONTEXT_CHUNKS_TOP_K': at_top_k_chunks, # The number of chunks requested for a question
        'RAG_SWITCH': True, # Front-end user input for enabling RAG
        'HISTORY_SWITCH': False, # Front-end user input for enabling history usage
        'BM25_SWITCH': True, # Front-end user input for enabling BM25 search
        'TOPIC_RETRIEVAL_SWITCH': True, # Front-end user input for enabling topic retrieval
        'HISTORIC_QUERY_SIMILARITY_THRESHOLD': 0.3
    }
}

history_window_size = 5

multi_shot_experiments_at_5_top_k = {

    # experiment_5: Naive response without retrieval and history
    'experiment_5' : {
        'DESIRED_HISTORY_WINDOW_SIZE': history_window_size, # The lookback window size for history utilization
        'DESIRED_CONTEXT_CHUNKS_TOP_K': at_top_k_chunks, # The number of chunks requested for a question
        'RAG_SWITCH': False, # Front-end user input for enabling RAG
        'HISTORY_SWITCH': True, # Front-end user input for enabling history usage
        'BM25_SWITCH': False, # Front-end user input for enabling BM25 search
        'TOPIC_RETRIEVAL_SWITCH': False, # Front-end user input for enabling topic retrieval
        'HISTORIC_QUERY_SIMILARITY_THRESHOLD': 0.3
    },

    # experiment_6: RAG response with basic retrieval and history
    'experiment_6' : {
        'DESIRED_HISTORY_WINDOW_SIZE': history_window_size, # The lookback window size for history utilization
        'DESIRED_CONTEXT_CHUNKS_TOP_K': at_top_k_chunks, # The number of chunks requested for a question
        'RAG_SWITCH': True, # Front-end user input for enabling RAG
        'HISTORY_SWITCH': True, # Front-end user input for enabling history usage
        'BM25_SWITCH': False, # Front-end user input for enabling BM25 search
        'TOPIC_RETRIEVAL_SWITCH': False, # Front-end user input for enabling topic retrieval
        'HISTORIC_QUERY_SIMILARITY_THRESHOLD': 0.3
    },

    # experiment_7: RAG response hybrid retrieval and history
    'experiment_7' : {
        'DESIRED_HISTORY_WINDOW_SIZE': history_window_size, # The lookback window size for history utilization
        'DESIRED_CONTEXT_CHUNKS_TOP_K': at_top_k_chunks, # The number of chunks requested for a question
        'RAG_SWITCH': True, # Front-end user input for enabling RAG
        'HISTORY_SWITCH': True, # Front-end user input for enabling history usage
        'BM25_SWITCH': True, # Front-end user input for enabling BM25 search
        'TOPIC_RETRIEVAL_SWITCH': False, # Front-end user input for enabling topic retrieval
        'HISTORIC_QUERY_SIMILARITY_THRESHOLD': 0.3
    },

    # experiment_8: RAG response hybrid with topic level retrieval and history
    'experiment_8' : {
        'DESIRED_HISTORY_WINDOW_SIZE': history_window_size, # The lookback window size for history utilization
        'DESIRED_CONTEXT_CHUNKS_TOP_K': at_top_k_chunks, # The number of chunks requested for a question
        'RAG_SWITCH': True, # Front-end user input for enabling RAG
        'HISTORY_SWITCH': True, # Front-end user input for enabling history usage
        'BM25_SWITCH': True, # Front-end user input for enabling BM25 search
        'TOPIC_RETRIEVAL_SWITCH': True, # Front-end user input for enabling topic retrieval
        'HISTORIC_QUERY_SIMILARITY_THRESHOLD': 0.3
    }
}

In [ ]:
def conduct_experiment(my_input_questions, experiment, experiment_type='zero_shot'):
    
    # Process each experiment in the selected experiment type
    for experiment_name, experiment_config in experiment.items():
        print(f"Running {experiment_name} with configuration: {experiment_config}")
        
        # Here you can customize the configuration further if needed (e.g., dynamic changes)
        process_list_of_queries_for_testing(
            saved_chats_topic_name=experiment_name, 
            list_of_questions=my_input_questions,
            task_folder='experiments',
            user_output_folder=experiment_type,
            DESIRED_HISTORY_WINDOW_SIZE=experiment_config['DESIRED_HISTORY_WINDOW_SIZE'],
            DESIRED_CONTEXT_CHUNKS_TOP_K=experiment_config['DESIRED_CONTEXT_CHUNKS_TOP_K'],
            RAG_SWITCH=experiment_config['RAG_SWITCH'],
            HISTORY_SWITCH=experiment_config['HISTORY_SWITCH'],
            BM25_SWITCH=experiment_config['BM25_SWITCH'],
            TOPIC_RETRIEVAL_SWITCH=experiment_config['TOPIC_RETRIEVAL_SWITCH'],
            HISTORIC_QUERY_SIMILARITY_THRESHOLD=experiment_config['HISTORIC_QUERY_SIMILARITY_THRESHOLD']
        )

In [ ]:
conduct_experiment(my_input_questions=my_input_questions,
                   experiment=zero_shot_experiments_at_5_top_k,
                   experiment_type='zero_shot'
)

Running experiment_1 with configuration: {'DESIRED_HISTORY_WINDOW_SIZE': 0, 'DESIRED_CONTEXT_CHUNKS_TOP_K': 5, 'RAG_SWITCH': False, 'HISTORY_SWITCH': False, 'BM25_SWITCH': False, 'TOPIC_RETRIEVAL_SWITCH': False, 'HISTORIC_QUERY_SIMILARITY_THRESHOLD': 0.3}
Processing query 1: Please explain batch normalization.

RAG Switch: False
History Switch: False
BM25 Switch: False
Topic Retrieval Switch: False

Response: Okay, I can certainly help you with that. Let's dive into the concept of batch normalization.

**1. Introductory Overview:**

Batch Normalization (BN) is a technique used in training artificial neural networks to improve the speed, performance, and stability of the learning process. Think of it like standardizing your data before feeding it into the neural network. By normalizing the activations within each batch of data, BN helps to mitigate the "internal covariate shift" problem, which occurs when the distribution of network activations changes during training. This often result

In [15]:
conduct_experiment(my_input_questions=my_input_questions,
                   experiment=multi_shot_experiments_at_5_top_k,
                   experiment_type='multi_shot'
)

Running experiment_5 with configuration: {'DESIRED_HISTORY_WINDOW_SIZE': 5, 'DESIRED_CONTEXT_CHUNKS_TOP_K': 5, 'RAG_SWITCH': False, 'HISTORY_SWITCH': True, 'BM25_SWITCH': False, 'TOPIC_RETRIEVAL_SWITCH': False, 'HISTORIC_QUERY_SIMILARITY_THRESHOLD': 0.3}
Processing query 1: Please explain batch normalization.

RAG Switch: False
History Switch: True
BM25 Switch: False
Topic Retrieval Switch: False

History was meant to be utilized, but the window_size is 0, so no lookback will be performed (window size == 0, no history requested).
Response: Okay, I can certainly explain batch normalization. Let's dive in!

### 1. Introductory Overview

Batch normalization (BN) is a technique used in training artificial neural networks to improve the speed, performance, and stability of the training process. At its core, batch normalization normalizes the activations of a layer in a neural network. This means it adjusts and scales the outputs of a layer to have a consistent mean and variance. Think of it

# Single Query Example: 

```python
# Collect user input and options dynamically from the front-end
# For demonstration, we're simulating user input here. In your front-end, these values will be collected from user interactions.

# Example user inputs (to be passed from the front-end)
DESIRED_HISTORY_WINDOW_SIZE = 3
DESIRED_CONTEXT_CHUNKS_TOP_K = 5

RAG_SWITCH = True  # Front-end user input for enabling RAG
HISTORY_SWITCH = False  # Front-end user input for enabling history usage
BM25_SWITCH = True  # Front-end user input for enabling BM25 search
TOPIC_RETRIEVAL_SWITCH = False  # Front-end user input for enabling topic retrieval

HISTORIC_QUERY_SIMILARITY_THRESHOLD = 0.3  # Front-end user input for similarity threshold

QUERY_TEXT = 'What is a neural radience field NERF?'
#QUERY_TEXT = input("Enter your question (or type 'exit' to quit): ")  # Get the query text from user input

# Your processing logic goes here (e.g., semantic search, BM25, RAG)
print(f"Processing query: {QUERY_TEXT}")
print(f"RAG Switch: {RAG_SWITCH}")
print(f"History Switch: {HISTORY_SWITCH}")
print(f"BM25 Switch: {BM25_SWITCH}")
print(f"Topic Retrieval Switch: {TOPIC_RETRIEVAL_SWITCH}")

# This incriment signals the initialization of a new query state placement matters
query_num += 1

# Call the `process_query()` function with the inputs
user_query_state_history[query_num] = process_query(
    DESIRED_HISTORY_WINDOW_SIZE, 
    DESIRED_CONTEXT_CHUNKS_TOP_K, 
    RAG_SWITCH, 
    HISTORY_SWITCH, 
    BM25_SWITCH, 
    TOPIC_RETRIEVAL_SWITCH, 
    HISTORIC_QUERY_SIMILARITY_THRESHOLD, 
    QUERY_TEXT, 
    user_query_state_history,
    query_num, 
    QDRANT_CLIENT, 
    CHUNK_COLLECTION,
    HISTORY_COLLECTION,
    LLM_MODEL,
    LLM_SYSTEM_PROMPT,
    DEVICE,
    EMBEDDING_MODEL, 
    TOKENIZER,
    BM25_SEARCH_FUNCTION
)

save_chat_json(user_query_state_history[query_num])

print(f"Processed query {query_num}: {QUERY_TEXT}")
print(f"Response: {user_query_state_history[query_num]['response_text']}")

# After processing the query, you might want to save the results to disk or perform further actions
save_chat_pkl_by_embedding(user_query_state_history, 
                            embedded_path='user_output/user_embedded_history.pkl',
                            non_embedded_path='user_output/user_non_embedded_history.pkl'
)

# Optionally, you could display or log the results of the query here for debugging purposes:
print(f"Processed query {query_num}: {QUERY_TEXT}")
print(f"Response: {user_query_state_history[query_num]['response_text']}")
```